In [7]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        #self.conv1_do = nn.Dropout(0.25) #--
        #self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        #self.conv2_do = nn.Dropout(p=0.25) #--
        #self.pool1 = nn.MaxPool2d(2, 2)
        #self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        #self.conv3_bn = nn.BatchNorm2d(128) #--
        #self.conv3_do = nn.Dropout(p=0.25) #--
        #self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        #self.conv4_do = nn.Dropout(p=0.25) #--
        #self.pool2 = nn.MaxPool2d(2, 2)
        #self.conv5 = nn.Conv2d(256, 512, 3)
        #self.conv5_do = nn.Dropout(p=0.25) #--
        #self.conv5_bn = nn.BatchNorm2d(512) #--
        #self.conv6 = nn.Conv2d(512, 1024, 3)
        #self.conv7 = nn.Conv2d(1024, 10, 3)
        #self.fc = nn.Linear(10, 10)

        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input Channel size -? OUtput channel size-? RFout-
        self.conv1_do = nn.Dropout(0.25) #--
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv2_do = nn.Dropout(p=0.25) #--
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv3_bn = nn.BatchNorm2d(128) #--
        self.conv3_do = nn.Dropout(p=0.25) #--
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv4_do = nn.Dropout(p=0.25) #--
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv5_do = nn.Dropout(p=0.25) #--
        self.conv5_bn = nn.BatchNorm2d(512) #--
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)
        self.fc = nn.Linear(10, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2_do(self.conv2(F.relu(self.conv1_do(self.conv1(x)))))))
        #x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = self.pool2(F.relu(self.conv4_do(self.conv4(F.relu(self.conv3_do(self.conv3_bn(self.conv3(x))))))))
        x = F.relu(self.conv6(F.relu(self.conv5_bn(self.conv5_do(self.conv5(x))))))
        #x = F.relu(self.conv6(F.relu(self.conv5_bn(self.conv5(x)))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        x = self.fc(x)
        return F.log_softmax(x)

In [15]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
           Dropout-2           [-1, 32, 28, 28]               0
            Conv2d-3           [-1, 64, 28, 28]          18,496
           Dropout-4           [-1, 64, 28, 28]               0
         MaxPool2d-5           [-1, 64, 14, 14]               0
            Conv2d-6          [-1, 128, 14, 14]          73,856
       BatchNorm2d-7          [-1, 128, 14, 14]             256
           Dropout-8          [-1, 128, 14, 14]               0
            Conv2d-9          [-1, 256, 14, 14]         295,168
          Dropout-10          [-1, 256, 14, 14]               0
        MaxPool2d-11            [-1, 256, 7, 7]               0
           Conv2d-12            [-1, 512, 5, 5]       1,180,160
          Dropout-13            [-1, 512, 5, 5]               0
      BatchNorm2d-14            [-1, 51

<ipython-input-14-a327bb9e2720>:50: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [19]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [20]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
#optimizer = optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-14-a327bb9e2720>:50: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=1.472564935684204 batch_id=45:  10%|▉         | 45/469 [02:41<25:49,  3.66s/it] 